<a href="https://colab.research.google.com/github/eridanletalis/GB_nn/blob/main/hw4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Загрузка библиотек и примеров

In [2]:
import cv2
from scipy import misc
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
from tensorflow.keras.models import Sequential, Model
from tensorflow.keras.layers import Conv2D, Input
from tensorflow.keras.layers import GlobalMaxPooling2D, MaxPooling2D, AveragePooling2D, GlobalAveragePooling2D 
from tensorflow.keras.datasets import mnist
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import models, layers
import tensorflow.keras as keras
from tensorflow.keras.datasets import fashion_mnist
from tensorflow.keras.layers import Dense, Flatten, Concatenate
from tensorflow.keras.utils import plot_model 
from __future__ import print_function
#import tensorflow.keras as keras # расскоментируйте эту строку, чтобы начать обучение
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import Dropout, Activation
import os



In [3]:
classes=['самолет', 'автомобиль', 'птица', 'кот', 'олень', 'собака', 'лягушка', 'лошадь', 'корабль', 'грузовик']


In [4]:
batch_size = 32
num_classes = 10
epochs = 5
data_augmentation = False
num_predictions = 20

In [5]:
# разделение тренировочной и тестовой выборки
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'тренировочные примеры')
print(x_test.shape[0], 'тестовые примеры')

# преобразование матрицы чисел 0-9 в бинарную матрицу чисел 0-1
y_train = keras.utils.to_categorical(y_train, num_classes)
y_test = keras.utils.to_categorical(y_test, num_classes)


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255



x_train shape: (50000, 32, 32, 3)
50000 тренировочные примеры
10000 тестовые примеры


In [5]:
x_train.shape[1:]

(32, 32, 3)

In [6]:
data_augmentation = True

In [8]:
from datetime import datetime
from tensorflow.keras import initializers

# конфигурирование слоев нейросети
model = Sequential()

# слои нейросети отвественные за свертку и max-pooling
# первый сверточный слой
model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                        kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid',
                        kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

model.add(layers.AveragePooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid', 
                        kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

model.add(layers.Flatten())

model.add(layers.Dense(84, activation='tanh', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

model.add(layers.Dense(10, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

# компиляция модели
model.compile(loss='categorical_crossentropy',
              optimizer='SGD',
              metrics=['accuracy'])



if not data_augmentation:
    print('Не используется data augmentation')
    model.fit(x_train, y_train,
              batch_size=batch_size,
              epochs=epochs,
              validation_data=(x_test, y_test),
              shuffle=True)
else:
    print('Использование data augmentation в реальном времени')
    # Препроцессинг и data augmentation в реальном времени:
    datagen = ImageDataGenerator(
        featurewise_center=False,
        samplewise_center=False,
        featurewise_std_normalization=False,
        samplewise_std_normalization=False,
        zca_whitening=False, 
        zca_epsilon=1e-06, 
        rotation_range=0, 
        width_shift_range=0.1,
        height_shift_range=0.1,
        shear_range=0., 
        zoom_range=0., 
        channel_shift_range=0.,
        fill_mode='nearest',
        cval=0.,
        horizontal_flip=True,
        vertical_flip=False,
        rescale=None,
        preprocessing_function=None,
        data_format=None,
        validation_split=0.0)

    # запуск data augmentation через fit
    #datagen.fit(x_train)

    # запуск data augmentation через fit_generator
    model.fit_generator(datagen.flow(x_train, y_train,
                                     batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)



# проверка работы обученной модели
scores = model.evaluate(x_test, y_test, verbose=1)
print('Test loss:', scores[0])
print('Test accuracy:', scores[1])

Использование data augmentation в реальном времени
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1563/1563 [==============================] - 67s 22ms/step - loss: 2.2192 - accuracy: 0.1699 - val_loss: 1.9325 - val_accuracy: 0.2862
Epoch 2/5
1563/1563 [==============================] - 34s 22ms/step - loss: 1.9396 - accuracy: 0.2912 - val_loss: 1.7962 - val_accuracy: 0.3501
Epoch 3/5
1563/1563 [==============================] - 34s 22ms/step - loss: 1.8467 - accuracy: 0.3237 - val_loss: 1.7650 - val_accuracy: 0.3605
Epoch 4/5
1563/1563 [==============================] - 34s 21ms/step - loss: 1.7896 - accuracy: 0.3492 - val_loss: 1.6964 - val_accuracy: 0.3797
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 1.6692 - accuracy: 0.3920
Test loss: 1.6692301034927368
Test accuracy: 0.3919999897480011


In [6]:
import pandas as pd

In [10]:
column = ['lenet-like_base', 
          'lenet-like_more_krnl_1','lenet-like_more_krnl_2', 'lenet-like_more_krnl_3', 
          'lenet-like_deep_1', 'lenet-like_deep_2', 'lenet-like_deep_3']

test_loss = pd.DataFrame(columns=column)
test_acc = pd.DataFrame(columns=column)

In [11]:
test_loss

,lenet-like_base,lenet-like_more_krnl_1,lenet-like_more_krnl_2,lenet-like_more_krnl_3,lenet-like_deep_1,lenet-like_deep_2,lenet-like_deep_3


# Эксперименты с простой сетью
Аналогично предыдущей работе будем выполнять работу по изменению числа слоёв и числа ядер, выполнять тестирование несколько раз и усреднять результаты.

1. lenet-like_base - базовая модель, относительно которой будут происходить расчёты
2. lenet-like_more_krnl_N - увеличение числа ядер
3. lenet-like_deep_N - увеличение числа слоёв

Далее всё будет сведено в таблицу, где будут получены статистики.


In [14]:
for i in range(1, 6):
  print(f"Базовая сеть, проход {i}")
  model = Sequential()
  model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid', 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Flatten())

  model.add(layers.Dense(84, activation='tanh', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Dense(10, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])
  print('Использование data augmentation в реальном времени')
  datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      zca_epsilon=1e-06, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0., 
      zoom_range=0., 
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0)

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[0]] = scores[0]
  test_acc.loc[i, column[0]] = scores[1]



  print(f"{column[1]}, проход {i}")
  model = Sequential()
  model.add(layers.Conv2D(12, kernel_size=(6, 6), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(24, kernel_size=(6, 6), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(240, kernel_size=(6, 6), strides=(1, 1), activation='tanh', padding='valid', 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Flatten())

  model.add(layers.Dense(95, activation='tanh', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Dense(10, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])
  print('Использование data augmentation в реальном времени')

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[1]] = scores[0]
  test_acc.loc[i, column[1]] = scores[1]


  print(f"{column[2]}, проход {i}")
  model = Sequential()
  model.add(layers.Conv2D(18, kernel_size=(6, 6), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(36, kernel_size=(6, 6), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(360, kernel_size=(6, 6), strides=(1, 1), activation='tanh', padding='valid', 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Flatten())

  model.add(layers.Dense(100, activation='tanh', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Dense(10, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])
  print('Использование data augmentation в реальном времени')

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[2]] = scores[0]
  test_acc.loc[i, column[2]] = scores[1]


  print(f"{column[3]}, проход {i}")
  model = Sequential()
  model.add(layers.Conv2D(32, kernel_size=(6, 6), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(64, kernel_size=(6, 6), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(512, kernel_size=(6, 6), strides=(1, 1), activation='tanh', padding='valid', 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Flatten())

  model.add(layers.Dense(120, activation='tanh', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Dense(10, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])
  print('Использование data augmentation в реальном времени')

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[3]] = scores[0]
  test_acc.loc[i, column[3]] = scores[1]


  print(f"{column[4]}, проход {i}")
  model = Sequential()
  model.add(layers.Conv2D(6, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(2, 2), padding='valid'))

  model.add(layers.Conv2D(120, kernel_size=(3, 3), strides=(1, 1), activation='tanh', padding='valid', 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Flatten())

  model.add(layers.Dense(84, activation='tanh', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Dense(10, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])
  print('Использование data augmentation в реальном времени')

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[4]] = scores[0]
  test_acc.loc[i, column[4]] = scores[1]


  print(f"{column[5]}, проход {i}")
  model = Sequential()
  model.add(layers.Conv2D(6, kernel_size=(32, 32), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(16, kernel_size=(16, 16), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  model.add(layers.Conv2D(32, kernel_size=(8, 8), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(64, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(120, kernel_size=(2, 2), strides=(1, 1), activation='tanh', padding='valid', 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))


  model.add(layers.Flatten())

  model.add(layers.Dense(84, activation='tanh', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Dense(10, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])
  print('Использование data augmentation в реальном времени')

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[5]] = scores[0]
  test_acc.loc[i, column[5]] = scores[1]




  print(f"{column[6]}, проход {i}")
  model = Sequential()
  model.add(layers.Conv2D(6, kernel_size=(16, 16), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(12, kernel_size=(5, 5), strides=(1, 1), activation='tanh', input_shape=x_train.shape[1:], padding="same", 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(16, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))

  model.add(layers.Conv2D(32, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(64, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid',
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))
  model.add(layers.MaxPooling2D(pool_size=(2, 2), strides=(1, 1), padding='valid'))
  model.add(layers.Conv2D(120, kernel_size=(5, 5), strides=(1, 1), activation='tanh', padding='valid', 
                          kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Flatten())

  model.add(layers.Dense(84, activation='tanh', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  model.add(layers.Dense(10, activation='softmax', kernel_initializer=initializers.RandomNormal(mean=0.0,stddev=0.15,seed=int(datetime.now().timestamp()))))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])
  print('Использование data augmentation в реальном времени')

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                        epochs=epochs,
                        validation_data=(x_test, y_test),
                        workers=4)
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[6]] = scores[0]
  test_acc.loc[i, column[6]] = scores[1]

Базовая сеть, проход 1
Использование data augmentation в реальном времени
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1563/1563 [==============================] - 34s 22ms/step - loss: 2.2410 - accuracy: 0.1589 - val_loss: 1.9148 - val_accuracy: 0.3051
Epoch 2/5
1563/1563 [==============================] - 34s 22ms/step - loss: 1.9284 - accuracy: 0.2967 - val_loss: 1.8239 - val_accuracy: 0.3371
Epoch 3/5
1563/1563 [==============================] - 33s 21ms/step - loss: 1.8395 - accuracy: 0.3234 - val_loss: 1.7843 - val_accuracy: 0.3395
Epoch 4/5
1563/1563 [==============================] - 34s 21ms/step - loss: 1.7888 - accuracy: 0.3409 - val_loss: 1.6907 - val_accuracy: 0.3879
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 1.7091 - accuracy: 0.3757
lenet-like_more_krnl_1, проход 1
Использование data augmentation в реальном времени
Epoch 1/5
1563/1563 [==============================] - 35s 22ms/step - loss: 2.2193 - accuracy: 0.1811 - val_loss: 1.9774 - val_accuracy: 0.2676
Epoch 2/5
1563/1563 [==============================] - 34s 22ms/step - loss: 1.9458 - accuracy: 0.2817

In [15]:
test_loss.astype(np.float64).describe()

,lenet-like_base,lenet-like_more_krnl_1,lenet-like_more_krnl_2,lenet-like_more_krnl_3,lenet-like_deep_1,lenet-like_deep_2,lenet-like_deep_3
count,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000,5.000000
mean,1.679351,1.721695,1.749617,1.842363,1.470135,1.813760,2.307909
std,0.038969,0.034882,0.021930,0.047057,0.031215,0.037846,0.002237
min,1.633312,1.686317,1.717043,1.787055,1.427989,1.767279,2.305252
25%,1.649793,1.688398,1.738445,1.794893,1.460475,1.785400,2.306739
50%,1.678357,1.731628,1.756040,1.873803,1.467606,1.814388,2.308011
75%,1.709074,1.732591,1.767949,1.875595,1.480790,1.848609,2.308248
max,1.726218,1.769542,1.768606,1.880470,1.513815,1.853126,2.311295


In [16]:
test_acc.astype(np.float64).describe()

,lenet-like_base,lenet-like_more_krnl_1,lenet-like_more_krnl_2,lenet-like_more_krnl_3,lenet-like_deep_1,lenet-like_deep_2,lenet-like_deep_3
count,5.00000,5.000000,5.000000,5.000000,5.00000,5.00000,5.000000
mean,0.38388,0.361960,0.356060,0.306960,0.46222,0.33752,0.100080
std,0.01297,0.015288,0.009175,0.015971,0.01300,0.01242,0.000192
min,0.37270,0.346100,0.346900,0.295000,0.44380,0.32250,0.099900
25%,0.37520,0.353400,0.348800,0.295100,0.45610,0.33220,0.100000
50%,0.37570,0.354000,0.353000,0.296600,0.46610,0.33340,0.100000
75%,0.39540,0.374800,0.365100,0.319300,0.46660,0.34490,0.100100
max,0.40040,0.381500,0.366500,0.328800,0.47850,0.35460,0.100400


Результаты мягко говоря не понятные. При увеличении ширины сети растут потери и уменьшается точность. При небольшом увеличении глубины сети (+1 слой) точность выросла, потеря уменьшилась, при дальнейшем увеличении глубины точность падает, потери растут. Возможно, более глубоким, чем 6ти слойным (4 Сonv + 1 Dense) сетям требуется большее число эпох, возможно глубокие слои не тренируются из-за падения градиента. С широкими сетями ситуация такая же - при увеличении количества ядер падает точность. Возможно, нужно больше эпох.

Попробуем провести эксперимент с другой сетью.

# Эксперимент со второй простой сетью

План эксперимента аналогичный.

In [7]:
column = ['lenet-like_base', 
          'lenet-like_more_krnl_1','lenet-like_more_krnl_2', 'lenet-like_more_krnl_3', 
          'lenet-like_deep_1', 'lenet-like_deep_2', 'lenet-like_deep_3']

test_loss = pd.DataFrame(columns=column)
test_acc = pd.DataFrame(columns=column)

In [8]:
test_loss

,lenet-like_base,lenet-like_more_krnl_1,lenet-like_more_krnl_2,lenet-like_more_krnl_3,lenet-like_deep_1,lenet-like_deep_2,lenet-like_deep_3


1. lenet-like_base - базовая модель, относительно которой будут происходить расчёты
2. lenet-like_more_krnl_N - увеличение числа ядер 
3. lenet-like_deep_N - увеличение числа слоёв (N=1 - уменьшение)

Далее всё будет сведено в таблицу, где будут получены статистики.

В этом эксперименте интересует стабильность обучения при одинаковых стартовых условиях, поэтому не будет изменяться ядро генератора псевдослучайных чисел.

Вариант с большим числом экспериментов не запустился - вышло доступное ОЗУ. Запустил с одним.


In [9]:
for i in range(1,2):
  print(f"Базовая сеть, проход {i}")
  model = Sequential()

  # слои нейросети отвественные за свертку и max-pooling
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( 0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  # полносвязные слои нейронной сети
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])



  print('Использование data augmentation в реальном времени')
  datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      zca_epsilon=1e-06, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0., 
      zoom_range=0., 
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0)

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      workers=4)


  # проверка работы обученной модели
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[0]] = scores[0]
  test_acc.loc[i, column[0]] = scores[1]


  print(f"{column[1]}, проход {i}")
  model = Sequential()

  # слои нейросети отвественные за свертку и max-pooling
  model.add(Conv2D(64, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( 0.25))

  model.add(Conv2D(128, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(128, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  # полносвязные слои нейронной сети
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])



  print('Использование data augmentation в реальном времени')
  datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      zca_epsilon=1e-06, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0., 
      zoom_range=0., 
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0)

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      workers=4)


  # проверка работы обученной модели
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[1]] = scores[0]
  test_acc.loc[i, column[1]] = scores[1]


  print(f"{column[2]}, проход {i}")
  model = Sequential()

  # слои нейросети отвественные за свертку и max-pooling
  model.add(Conv2D(128, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(128, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( 0.25))

  model.add(Conv2D(256, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(256, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  # полносвязные слои нейронной сети
  model.add(Flatten())
  model.add(Dense(1024))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])



  print('Использование data augmentation в реальном времени')
  datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      zca_epsilon=1e-06, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0., 
      zoom_range=0., 
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0)

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      workers=4)


  # проверка работы обученной модели
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[2]] = scores[0]
  test_acc.loc[i, column[2]] = scores[1]


  print(f"{column[3]}, проход {i}")
  model = Sequential()

  # слои нейросети отвественные за свертку и max-pooling
  model.add(Conv2D(256, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(256, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( 0.25))

  model.add(Conv2D(512, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(512, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  # полносвязные слои нейронной сети
  model.add(Flatten())
  model.add(Dense(2048))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])



  print('Использование data augmentation в реальном времени')
  datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      zca_epsilon=1e-06, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0., 
      zoom_range=0., 
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0)

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      workers=4)


  # проверка работы обученной модели
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[3]] = scores[0]
  test_acc.loc[i, column[3]] = scores[1]


  print(f"{column[4]}, проход {i}")
  model = Sequential()

  # слои нейросети отвественные за свертку и max-pooling
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( 0.25))


  # полносвязные слои нейронной сети
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])



  print('Использование data augmentation в реальном времени')
  datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      zca_epsilon=1e-06, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0., 
      zoom_range=0., 
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0)

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      workers=4)


  # проверка работы обученной модели
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[4]] = scores[0]
  test_acc.loc[i, column[4]] = scores[1]


  print(f"{column[5]}, проход {i}")
  model = Sequential()

  # слои нейросети отвественные за свертку и max-pooling
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( 0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  # полносвязные слои нейронной сети
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])



  print('Использование data augmentation в реальном времени')
  datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      zca_epsilon=1e-06, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0., 
      zoom_range=0., 
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0)

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      workers=4)


  # проверка работы обученной модели
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[5]] = scores[0]
  test_acc.loc[i, column[5]] = scores[1]

  print(f"{column[6]}, проход {i}")
  model = Sequential()

  # слои нейросети отвественные за свертку и max-pooling
  model.add(Conv2D(32, (3, 3), padding='same',
                  input_shape=x_train.shape[1:]))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(1, 1)))
  model.add(Dropout( 0.25))

  model.add(Conv2D(32, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(32, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout( 0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  model.add(Conv2D(64, (3, 3), padding='same'))
  model.add(Activation('relu'))
  model.add(Conv2D(64, (3, 3)))
  model.add(Activation('relu'))
  model.add(MaxPooling2D(pool_size=(2, 2)))
  model.add(Dropout(0.25))

  # полносвязные слои нейронной сети
  model.add(Flatten())
  model.add(Dense(512))
  model.add(Activation('relu'))
  model.add(Dropout(0.5))
  model.add(Dense(num_classes))
  model.add(Activation('softmax'))

  # компиляция модели
  model.compile(loss='categorical_crossentropy',
                optimizer='SGD',
                metrics=['accuracy'])



  print('Использование data augmentation в реальном времени')
  datagen = ImageDataGenerator(
      featurewise_center=False,
      samplewise_center=False,
      featurewise_std_normalization=False,
      samplewise_std_normalization=False,
      zca_whitening=False, 
      zca_epsilon=1e-06, 
      rotation_range=0, 
      width_shift_range=0.1,
      height_shift_range=0.1,
      shear_range=0., 
      zoom_range=0., 
      channel_shift_range=0.,
      fill_mode='nearest',
      cval=0.,
      horizontal_flip=True,
      vertical_flip=False,
      rescale=None,
      preprocessing_function=None,
      data_format=None,
      validation_split=0.0)

  model.fit_generator(datagen.flow(x_train, y_train,
                                    batch_size=batch_size),
                      epochs=epochs,
                      validation_data=(x_test, y_test),
                      workers=4)


  # проверка работы обученной модели
  scores = model.evaluate(x_test, y_test, verbose=1)
  test_loss.loc[i, column[6]] = scores[0]
  test_acc.loc[i, column[6]] = scores[1]

Базовая сеть, проход 1
Использование data augmentation в реальном времени
Epoch 1/5


/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/training.py:1844: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


1563/1563 [==============================] - 65s 21ms/step - loss: 2.2196 - accuracy: 0.1553 - val_loss: 1.9394 - val_accuracy: 0.3020
Epoch 2/5
1563/1563 [==============================] - 32s 21ms/step - loss: 1.9552 - accuracy: 0.2841 - val_loss: 1.6869 - val_accuracy: 0.3957
Epoch 3/5
1563/1563 [==============================] - 33s 21ms/step - loss: 1.7512 - accuracy: 0.3584 - val_loss: 1.5259 - val_accuracy: 0.4424
Epoch 4/5
1563/1563 [==============================] - 33s 21ms/step - loss: 1.6368 - accuracy: 0.4017 - val_loss: 1.4241 - val_accuracy: 0.4780
Epoch 5/5
313/313 [==============================] - 1s 3ms/step - loss: 1.3464 - accuracy: 0.5155
lenet-like_more_krnl_1, проход 1
Использование data augmentation в реальном времени
Epoch 1/5
1563/1563 [==============================] - 35s 22ms/step - loss: 2.2001 - accuracy: 0.1699 - val_loss: 1.8637 - val_accuracy: 0.3515
Epoch 2/5
1563/1563 [==============================] - 34s 22ms/step - loss: 1.8712 - accuracy: 0.3210

In [10]:
test_loss.astype(np.float64).describe()

,lenet-like_base,lenet-like_more_krnl_1,lenet-like_more_krnl_2,lenet-like_more_krnl_3,lenet-like_deep_1,lenet-like_deep_2,lenet-like_deep_3
count,1.000000,1.00000,1.00000,1.000000,1.000000,1.000000,1.000000
mean,1.346444,1.28954,1.23719,1.169845,1.266097,1.477066,1.582013
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,1.346444,1.28954,1.23719,1.169845,1.266097,1.477066,1.582013
25%,1.346444,1.28954,1.23719,1.169845,1.266097,1.477066,1.582013
50%,1.346444,1.28954,1.23719,1.169845,1.266097,1.477066,1.582013
75%,1.346444,1.28954,1.23719,1.169845,1.266097,1.477066,1.582013
max,1.346444,1.28954,1.23719,1.169845,1.266097,1.477066,1.582013


In [11]:
test_acc.astype(np.float64).describe()

,lenet-like_base,lenet-like_more_krnl_1,lenet-like_more_krnl_2,lenet-like_more_krnl_3,lenet-like_deep_1,lenet-like_deep_2,lenet-like_deep_3
count,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000,1.0000
mean,0.5155,0.5319,0.5538,0.5886,0.5461,0.4609,0.4176
std,NaN,NaN,NaN,NaN,NaN,NaN,NaN
min,0.5155,0.5319,0.5538,0.5886,0.5461,0.4609,0.4176
25%,0.5155,0.5319,0.5538,0.5886,0.5461,0.4609,0.4176
50%,0.5155,0.5319,0.5538,0.5886,0.5461,0.4609,0.4176
75%,0.5155,0.5319,0.5538,0.5886,0.5461,0.4609,0.4176
max,0.5155,0.5319,0.5538,0.5886,0.5461,0.4609,0.4176


# Вывод по второму эксперименту
Для этого эксперимента очевидно, что расширение сети делает её результаты гораздо точнее, чем увеличение глубины - при увеличении глубины точность резко падает (возможно, не хватает количества эпох), при уменьшении глубины точность возрастает.

Таким образом, более широкие сети предпочтительнее более глубоких при равном датасете и числе эпох обучения.


# "Сложный" эксперимент 


Эксперимент со сложной сетью в другом блокноте
https://github.com/eridanletalis/GB_nn/blob/main/hw4_2.ipynb